In [2]:
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Conv2DTranspose, Concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K
import cv2
import os

In [3]:
EPOCHS = 150
PATIENCE = 50
BATCH_SIZE = 16
OPTIMIZER = 'adam'
DIM = 512
MODEL_NAME = "best_gridsearch_unet_mesh.h5"
CATEGORY = "mesh"

tf.random.set_seed(1)

src_arr = "saved_arrays/" + CATEGORY # local pc


In [4]:

#Load train, validation and test data. The images are augmented in train and validation. For each image was created a random modified new image.
#Images are normalized in [0,1].
train_images = np.load(src_arr + "/" + CATEGORY + "_train_images.npy")[:840]
train_images = train_images/255.0

train_masks = np.load(src_arr + "/" + CATEGORY + "_train_masks.npy")[:840]
train_masks = np.expand_dims(train_masks, axis=-1)

val_images = np.load(src_arr + "/" + CATEGORY + "_val_images.npy")[159
val_images = val_images/255.0

val_masks = np.load(src_arr + "/" + CATEGORY + "_val_masks.npy")
val_masks = np.expand_dims(val_masks, axis=-1)

test_images = np.load(src_arr + "/" + CATEGORY + "_test_images.npy")
test_images = test_images/255.0

test_masks = np.load(src_arr + "/" + CATEGORY + "_test_masks.npy")
test_masks = np.expand_dims(test_masks, axis=-1)

print(train_images.shape)
print(val_masks.shape)

(854, 512, 512, 3)
(163, 512, 512, 1)


In [5]:
#during training
def mean_iou(y_true, y_pred):
    yt0 = y_true[:,:,:,0]
    yp0 = K.cast(y_pred[:,:,:,0] > 0.5, 'float32')
    inter = tf.math.count_nonzero(tf.logical_and(tf.equal(yt0, 1), tf.equal(yp0, 1)))
    union = tf.math.count_nonzero(tf.add(yt0, yp0))
    iou = tf.where(tf.equal(union, 0), 1., tf.cast(inter/union, 'float32'))
    return iou

In [6]:

def unet(optimizer, loss, sz = (DIM, DIM, 3)):
    x = Input(sz)
    inputs = x
  
    #down sampling 
    f = 8
    layers = []
  
    for i in range(0, 6):
        x = Conv2D(f, 3, activation='relu', padding='same') (x)
        x = Conv2D(f, 3, activation='relu', padding='same') (x)
        layers.append(x)
        x = MaxPooling2D() (x)
        f = f*2
    ff2 = 64 
  
    #bottleneck 
    j = len(layers) - 1
    x = Conv2D(f, 3, activation='relu', padding='same') (x)
    x = Conv2D(f, 3, activation='relu', padding='same') (x)
    x = Conv2DTranspose(ff2, 2, strides=(2, 2), padding='same') (x)
    x = Concatenate(axis=3)([x, layers[j]])
    j = j -1 
  
    #upsampling 
    for i in range(0, 5):
        ff2 = ff2//2
        f = f // 2 
        x = Conv2D(f, 3, activation='relu', padding='same') (x)
        x = Conv2D(f, 3, activation='relu', padding='same') (x)
        x = Conv2DTranspose(ff2, 2, strides=(2, 2), padding='same') (x)
        x = Concatenate(axis=3)([x, layers[j]])
        j = j -1 

    #classification 
    x = Conv2D(f, 3, activation='relu', padding='same') (x)
    x = Conv2D(f, 3, activation='relu', padding='same') (x)
    outputs = Conv2D(1, 1, activation='sigmoid') (x)
  
    #model creation 
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer = optimizer, loss = loss, metrics = [mean_iou])
    model.summary()

    return model


In [7]:
def binary_focal_loss(gamma=2., alpha=.25):
   
    def binary_focal_loss_fixed(y_true, y_pred):
       
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed

In [8]:

def plot_img_and_masks(raw, mask):
    # image + prediction mask + target mask
    pred = model.predict(np.expand_dims(raw, 0))
    pred_msk = pred.squeeze()
    pred_msk = np.stack((pred_msk,) * 3, axis=-1)
    pred_msk[pred_msk >= 0.5] = 1
    pred_msk[pred_msk < 0.5] = 0
    target_msk = np.stack((mask,) * 3, axis=-1)
    raw = np.float32(raw)
    raw = cv2.cvtColor(raw, cv2.COLOR_RGB2BGR)
    combined = np.concatenate([raw, pred_msk, target_msk], axis=1)
    return combined

In [9]:
def quantitative_measures(pred_masks):
    test_masks = np.load(src_arr + "/" + CATEGORY + "_test_masks.npy")
    test_masks = np.expand_dims(test_masks, axis=-1)
    print("TEST RESULTS")

    iou = mean_iou_test(test_masks, pred_masks)
    print("mean iou", iou)
    dice = dice_coeff(test_masks, pred_masks)
    print("dice coeff.", dice)
    acc = pixel_accuracy(test_masks, pred_masks)
    print("pixel acc.", acc)
    acc_1 = pixel_accuracy_class1(test_masks, pred_masks)
    print("pixel acc. for " + CATEGORY + " only", acc_1)
    
    return iou, dice, acc, acc_1


In [10]:
import itertools


optimizers = ["adam", "adadelta", "sgd", "adagrad", "rmsprop"]
losses = [binary_focal_loss(gamma=2.0, alpha=0.25), "binary_crossentropy"]
#strides = [(2,2), (3,3)]
#paddings = ["same", "valid" ]
combinations = list(itertools.product(optimizers, losses))



In [ ]:
result_list = []
for optimizer, loss in combinations:
    print("Optimizer:", optimizer, "Loss function:", loss)
    model = unet(optimizer, loss, sz=(DIM,DIM,3))
    history  = model.fit(train_images, train_masks, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(val_images, val_masks))
    pred_masks = model.predict(test_images)
    iou, dice, acc, acc_1 = quantitative_measures(pred_masks)
    result = [optimizer, loss, history.history, iou, dice, acc, acc_1]
    result_list.append(result)
    
   

Optimizer: adam Loss function: <function binary_focal_loss.<locals>.binary_focal_loss_fixed at 0x7f2f27721950>


2023-02-01 17:22:08.624379: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-01 17:22:08.699244: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-01 17:22:08.699536: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-01 17:22:08.701172: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 8)  224         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 8)  584         conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 256, 256, 8)  0           conv2d_1[0][0]                   
______________________________________________________________________________________________

2023-02-01 17:22:12.440878: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2686451712 exceeds 10% of free system memory.
2023-02-01 17:22:15.774548: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2686451712 exceeds 10% of free system memory.
2023-02-01 17:22:18.008901: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/150


...

54/54 [==============================] - ETA: 0s - loss: 0.0703 - mean_iou: 0.0639

2023-02-01 17:22:41.903141: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 512753664 exceeds 10% of free system memory.
2023-02-01 17:22:43.215981: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 512753664 exceeds 10% of free system memory.


54/54 [==============================] - 27s 217ms/step - loss: 0.0703 - mean_iou: 0.0639 - val_loss: 0.0710 - val_mean_iou: 0.0000e+00
Epoch 2/150
54/54 [==============================] - 7s 125ms/step - loss: 0.0657 - mean_iou: 0.2501 - val_loss: 0.0633 - val_mean_iou: 0.2754
Epoch 3/150
54/54 [==============================] - 7s 124ms/step - loss: 0.0603 - mean_iou: 0.5826 - val_loss: 0.0597 - val_mean_iou: 0.6259
Epoch 4/150
54/54 [==============================] - 7s 138ms/step - loss: 0.0564 - mean_iou: 0.6119 - val_loss: 0.0549 - val_mean_iou: 0.5064
Epoch 5/150
54/54 [==============================] - 7s 135ms/step - loss: 0.0541 - mean_iou: 0.5491 - val_loss: 0.0632 - val_mean_iou: 0.6327
Epoch 6/150
54/54 [==============================] - 7s 125ms/step - loss: 0.0546 - mean_iou: 0.6077 - val_loss: 0.0566 - val_mean_iou: 0.6528
Epoch 7/150
54/54 [==============================] - 7s 125ms/step - loss: 0.0492 - mean_iou: 0.5962 - val_loss: 0.0534 - val_mean_iou: 0.6254
Epoch 

2023-02-01 17:24:05.896240: W tensorflow/core/framework/op_kernel.cc:1692] OP_REQUIRES failed at gather_op.cc:158 : Resource exhausted: OOM when allocating tensor with shape[16,512,512,3] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
2023-02-01 17:24:06.211351: W tensorflow/core/framework/op_kernel.cc:1692] OP_REQUIRES failed at gather_op.cc:158 : Resource exhausted: OOM when allocating tensor with shape[16,512,512,3] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu


43/54 [======================>.......] - ETA: 1s - loss: 0.0478 - mean_iou: 0.6579

In [ ]:

# Write the results to a .txt file
with open('combination_results.txt', 'w') as f:
    for result in result_list:
        f.write("Optimizer: " + result[0] + ", Loss function: " + str(result[1]) + ", quantitative results :  iou, dice, acc, acc_1 " + "\n")
        for i in range(EPOCHS):
            f.write("Epoch " + str(i+1) + ": " + "loss: " + str(result[2]['loss'][i]) + ", val_loss: " + str(result[2]['val_loss'][i]) 
                    + ", iou " + str(result[3]) + ",dice " + + str(result[4])  + ",acc " +  str(result[5]) + ",acc_1 " +  str(result[6]) + "\n"+"\n"+"\n"+"\n")

